#### SAS宏

----

##### 第一部分:简单回顾

```
DATA ACCT_C03;
SET ACCT(WHERE=(MONTH_NBR = "352")); ...
RUN;
DATA EVENT_C03;
SET EVENT(WHERE=(MONTH_NBR = "352")); ...
RUN;
DATA ACCT_M201902;
SET ACCT;
WHERE START_DT<="28FEB2019"D AND END_DT>"28FEB2019"D;
...
RUN;
DATA ACCT_M201903;
SET ACCT;
WHERE START_DT<="31MAR2019"D AND END_DT>"31MAR2019"D; ...
RUN;
```

##### 第二部分 SAS宏

(1)宏概述

• 宏实现完成重复任务减少必要代码量，对代码的模块化封装，使程序易读、便于修改、移植、方便重复使用;

• 宏包含两部分:宏命令和宏变量;

• 宏命令通常加"%"作为前缀，宏变量通常加"&"作为前缀;

(2)宏变量

用%LET创建一个宏变量，基本形式: 

%LET VARNAME = VALUE; 

&VARNAME.

• 宏变量有局部宏变量和全局宏变量;

• 如果在宏的内部定义则为局部宏变量，只能在内部使用; 

• 如果在开放代码中定义则为全局宏变量;

(3)宏变量声明方法

• %LET语句

• DATA步中使用CALL SYMPUT

• PROC SQL中使用SELECT INTO:

(4)宏结构

宏可以使一段代码在一个或多个程序中被反复使用，而不需要重复的 去编写相同或相似的代码；

宏的基本形式: 

```
%MACRO MACRO_NAME;
    MACRO_CODE; 
%MEND MACRO_NAME;

%MACRO_NAME；
```

宏语句中，可以在宏名称后的括号内列出宏参数的名字 基本形式为:

```
%MACRO MACRO_NAME(PARA_1=,PARA_2,...,PARA_N=);
    MACRO_CODE; 
%MEND MACRO_NAME;

%MACRO_NAME(PARA_1=,PARA_2,...,PARA_N=);
```

(5)宏语句

• 宏的条件语句

基本形式为:

```
%IF condition %THEN action; 
%ELSE %IF condition %THEN action; 
%ELSE action;
```

• 宏的控制语句

基本形式为:

```
%DO ... %TO ...; action; %END;
%DO %WHILE(condition); action; %END; 
%DO %UNTIL(condition); action; %END;
```

(6)内置函数

• %SYSFUNC

调用所有SAS的内置函数；

```
%IF %SYSFUNC(MOD(&I., 2)) = 1 
%THEN action;
```

• %EVAL

对表达式进行数值计算；

```
%LET S = 0;
%LET S = %EVAL(&S. + &I.);
```
----
MORE CODE

In [1]:
OPTIONS COMPRESS = YES;

SAS Connection established. Subprocess id is 17785



In [2]:
DATA CARS;
KEEP MAKE MODEL MSRP;
SET SASHELP.CARS;
RUN;

In [3]:
DATA DVAR01;
SET CARS;
WHERE MAKE = "BMW";
RUN;

DATA DVAR02;
SET CARS;
WHERE MAKE = "BMW";
RUN;

DATA DVAR03;
SET CARS;
WHERE MAKE = "BMW";
RUN;

In [4]:
%LET VAR1 = BMW;

DATA DVAR01;
SET CARS;
WHERE MAKE = "&VAR1.";
RUN;

DATA DVAR02;
SET CARS;
WHERE MAKE = "&VAR1.";
RUN;

DATA DVAR03;
SET CARS;
WHERE MAKE = "&VAR1.";
RUN;

/* ... */

%PUT &VAR1.;

In [5]:
/* 1 */
%LET VAR1 = Acura;
%PUT &VAR1.;

DATA DVAR1;
SET CARS;
WHERE MAKE = "&VAR1.";
RUN;


/* 2 */
DATA _NULL_;
    CALL SYMPUT("VAR2", "Acura");
RUN;
%PUT &VAR2.;

DATA DVAR2;
SET CARS;
WHERE MAKE = "&VAR2.";
RUN;


/* 3 */
PROC SQL NOPRINT;
SELECT 
    MAKE INTO: VAR3
FROM CARS;
QUIT;
%PUT &VAR3.;

PROC SQL NOPRINT;
SELECT 
    MAX(MAKE) INTO :VAR3
FROM CARS;
QUIT;
%PUT &VAR3.;

PROC SQL NOPRINT;
SELECT 
    MAX(MAKE),MIN(MSRP) INTO :VAR3, :VAR4 
FROM CARS;
QUIT;
%PUT &VAR3. &VAR4.;

DATA DVAR3;
SET CARS;
WHERE MAKE = "&VAR3.";
RUN;

In [6]:
%MACRO T1;
    DATA A;
    A = 1;
    RUN;
%MEND T1;
%T1;

In [7]:
%MACRO T2(V=,);
    %PUT &V.;

    DATA DVAR_&V.;
    SET CARS;
    WHERE MAKE = "&V.";
    RUN;
%MEND T2;

%T2(V=BMW);
%T2(V=Acura);

In [8]:
%MACRO T2(V=,);
    %PUT &V.;

    DATA DVAR_&V.;
    SET CARS;
    WHERE MAKE = "&V.";
    RUN;
%MEND T2;

%PUT &V.;
%T2(V=BMW);
%PUT &V.;

In [9]:
%LET M = BMW;

%MACRO T3(V=,);
    %PUT &V.;
    
    %IF &V.= &M. %THEN %DO;
        DATA DVAR_&V.;
        SET CARS;
        WHERE MAKE = "&V.";
        RUN;
        /* ... */
    %END;
    %ELSE %DO;
        DATA DVAR_&V.;
        SET CARS;
        WHERE MAKE = "&V." AND MSRP > 50000;
        RUN;
        /* ... */
    %END;

%MEND T3;

%T3(V=BMW);
%T3(V=Acura);

In [10]:
DATA A;
DO I = 1 TO 10;
    T = I + 2;
    OUTPUT;
END;
RUN;

PROC PRINT DATA = A;
RUN;

Obs,I,T
1,1,3
2,2,4
3,3,5
4,4,6
5,5,7
6,6,8
7,7,9
8,8,10
9,9,11
10,10,12


In [12]:
DATA ACCT;
INPUT SEX $ MONTH;
CARDS;
X0001 0
X0001 1
X0001 2
X0001 3
X0001 4
X0002 0
X0002 1
X0003 9
X0004 2
X0005 9
;
RUN;

%MACRO T4(V=,);
    %PUT &V.;
    
    %DO I = 0 %TO &V.;
        DATA A_&I.;
        SET ACCT;
        WHERE MONTH = &I.;
        RUN;
    %END;
%MEND T4;

%T4(V=10);

In [20]:
%MACRO T5(V=, N=10000);
    %PUT &V.;
    
    DATA MSRP;
    SET CARS;
    IMSRP = ROUNDZ(MSRP, &N.);
    %DO I = 0 %TO &V.;
        IF MSRP > &I.*&N. THEN A&I. = 1;ELSE A&I. = 0;
    %END;
    DROP MAKE MODEL;
    RUN;
    
    PROC PRINT DATA = MSRP(OBS=&V.);
    RUN;
%MEND T5;

%T5(V=6);

Obs,MSRP,IMSRP,A0,A1,A2,A3,A4,A5,A6
1,"$36,945",40000,1,1,1,1,0,0,0
2,"$23,820",20000,1,1,1,0,0,0,0
3,"$26,990",30000,1,1,1,0,0,0,0
4,"$33,195",30000,1,1,1,1,0,0,0
5,"$43,755",40000,1,1,1,1,1,0,0
6,"$46,100",50000,1,1,1,1,1,0,0


In [21]:
%LET S = 1;
%LET I = 10;
%PUT &S. &I.;


%LET S = &S. + &I.;
%PUT &S. &I.;


%LET S = 1;
%LET I = 10;
%LET S = %EVAL(&S. + &I.);
%PUT &S. &I.;